<a href="https://colab.research.google.com/github/wkd-woo/finance-statements/blob/main/%EC%86%90%EC%9D%B5%EC%9D%BC%EA%B4%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U finance-datareader

     |████████████████████████████████| 48 kB 2.7 MB/s 


In [2]:
import pandas as pd
import os
from tqdm import tqdm
import FinanceDataReader as fdr

# 파일별로 분류

In [3]:
import glob

재무상태표 = [] # _01_
손익계산서 = [] # _02_
포괄손익계산서 = [] # _03_
현금흐름표 = [] # _04_
자본변동표 = [] #_05_


for name in glob.glob('/content/drive/MyDrive/22캡스톤디자인/재무일괄/*.txt'):
  if '_01_' in name:
    재무상태표.append(name)
  elif '_02_' in name:
    손익계산서.append(name)
  elif '_03_' in name:
    포괄손익계산서.append(name)
  elif '_04_' in name:
    현금흐름표.append(name)
  elif '_05_' in name:
    자본변동표.append(name)

In [4]:
sample = pd.DataFrame(columns=['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', '당기'])

# 데이터 추출

In [5]:
for path in tqdm(손익계산서):
  df = pd.read_csv(path, delimiter = '\t', encoding='cp949')

  당기 = ''
  if df['보고서종류'][0] == '1분기보고서':
    당기 = '당기 1분기 누적' 
  elif df['보고서종류'][0] == '반기보고서':
    당기 = '당기 반기 누적' 
  elif df['보고서종류'][0] == '3분기보고서':
    당기 = '당기 3분기 누적' 
  elif df['보고서종류'][0] == '사업보고서':
    당기 = '당기' 

  df['사업연도'] = path[52:56] # 사업연도 컬럼 d설정 - 파일명에서 가져온다
  df['항목명'] = df['항목명'].str.strip() # 항목명 컬럼 문자열 전처리
  df['항목코드'] = df['항목코드'].str.strip() # 항목코드 컬럼 문자열 전처리

  df = df[['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', 당기]]

  전체종목 = len(df['종목코드'].unique())

  # (수익)매출액 (ifrs-full_Revenue)
  매출액 = df[(df['항목코드'] == 'ifrs_Revenue') | (df['항목코드'] == 'ifrs-full_Revenue') | (df['항목명'].isin(['매출액','(수익)매출액','매출액(수익)','수익(매출액)']))]
  매출액.rename(columns={당기:"당기"}, inplace=True)
  매출액.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  sample = sample.append(매출액, sort=False, ignore_index=True)


  # 매출총이익 (ifrs-full_GrossProfit))
  매출총이익 = df[(df['항목코드'] == 'ifrs_GrossProfit') | (df['항목코드'] == 'ifrs-full_GrossProfit')| (df['항목명'].isin(['매출총이익','(수익)매출총이익','매출총이익(손실)']))]
  매출총이익.rename(columns={당기:"당기"}, inplace=True)
  매출총이익.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  sample = sample.append(매출총이익, sort=False, ignore_index=True)

  # 영업이익(손실) (dart_OperatingIncomeLoss)
  영업이익 = df[(df['항목코드'] == 'dart_OperatingIncomeLoss')|(df['항목명'].isin(['영업이익','영업이익(손실)']))]
  영업이익.rename(columns={당기:"당기"}, inplace=True)
  영업이익.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  sample = sample.append(영업이익, sort=False, ignore_index=True)
  
  # 당기순이익(손실) (ifrs_ProfitLoss)
  당기순이익 = df[(df['항목코드'] == 'ifrs-full_ProfitLoss') | (df['항목코드'] == 'ifrs_ProfitLoss') |  (df['항목명'].isin(['당기순이익','당기순이익(손실)']))]
  당기순이익.rename(columns={당기:"당기"}, inplace=True)
  당기순이익.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  sample = sample.append(당기순이익, sort=False, ignore_index=True)

  # 비지배지분(ifrs_ProfitLossAttributableToNoncontrollingInterests)
  비지배지분 = df[(df['항목코드'] == 'ifrs-full_ProfitLossAttributableToNoncontrollingInterests') | (df['항목코드'] == 'ifrs_ProfitLossAttributableToNoncontrollingInterests') | (df['항목명'].str.contains('비지배'))]
  비지배지분.rename(columns={당기:"당기"}, inplace=True)
  비지배지분.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  sample = sample.append(비지배지분, sort=False, ignore_index=True)

  # 감가상각비(dart_DepreciationExpense)
  감가상각비 = df[(df['항목코드'] == 'dart_DepreciationExpense')|(df['항목명'].isin(['감가상각비']))]
  감가상각비.rename(columns={당기:"당기"}, inplace=True)
  sample = sample.append(감가상각비, sort=False, ignore_index=True)

  # 법인세비용차감전순이익(손실)(ifrs-full_ProfitLossBeforeTax)
  법인세차감전순이익 = df[(df['항목코드'] == 'ifrs-full_ProfitLossBeforeTax') | (df['항목코드'] == 'ifrs_ProfitLossBeforeTax') | df['항목명'].isin(['법인세차감전순이익','법인세차감전순이익(손실)'])]
  법인세차감전순이익.rename(columns={당기:"당기"}, inplace=True)
  법인세차감전순이익.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  sample = sample.append(법인세차감전순이익, sort=False, ignore_index=True)
  


  0%|          | 0/24 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
100%|██████████| 24/24 [00:10<00:00,  2.20it/s]


# 데이터 전처리

In [6]:
sample['종목코드'] = sample['종목코드'].apply(lambda x:x[1:-1]) 

In [26]:
mask = (sample['항목명'].str.contains('포괄|계속|중단'))|(sample['항목코드'].str.contains('continu|Continu'))
sample = sample[~mask]

In [23]:
len(sample['항목코드'].unique())

68

In [9]:
sample['항목명'] = sample.apply(lambda x: '매출액' if x['항목코드'] == 'ifrs_Revenue' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '매출액' if x['항목코드'] == 'ifrs-full_Revenue' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '매출총이익' if x['항목코드'] == 'ifrs_GrossProfit' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '매출총이익' if x['항목코드'] == 'ifrs-full_GrossProfit' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '영업이익' if x['항목코드'] == 'dart_OperatingIncomeLoss' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '당기순이익' if x['항목코드'] == 'ifrs_ProfitLoss' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '당기순이익' if x['항목코드'] == 'ifrs-full_ProfitLoss' else x['항목명'], axis=1)

sample['항목명'] = sample.apply(lambda x: '당기순이익(비지배)' if 'ProfitLossAttributableToNoncontrollingInterests' in x['항목코드'] else x['항목명'], axis=1)

sample['항목명'] = sample.apply(lambda x: '감가상각비' if x['항목코드'] == 'dart_DepreciationExpense' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '법인세비용차감전순이익' if x['항목코드'] == 'ifrs_ProfitLossBeforeTax' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '법인세비용차감전순이익' if x['항목코드'] == 'ifrs-full_ProfitLossBeforeTax' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '매출액' if x['항목명'] == '수익(매출액)' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '매출총이익' if x['항목명'] == '매출총이익(손실)' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '영업이익' if x['항목명'] == '영업이익(손실)' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '당기순이익' if x['항목명'] == '당기순이익(손실)' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '법인세차감전순이익' if x['항목명'] == '법인세차감전순이익(손실)' else x['항목명'], axis=1)
sample['항목명'] = sample.apply(lambda x: '법인세차감전순이익' if x['항목명'] == '법인세비용차감전순이익' else x['항목명'], axis=1)

sample

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,001040,2016,사업보고서,CJ,715,ifrs_Revenue,매출액,"23,954,197,397,000"
1,000120,2016,사업보고서,CJ대한통운,493,ifrs_Revenue,매출액,"6,081,945,642,204"
2,155660,2016,사업보고서,DSR,139,ifrs_Revenue,매출액,"201,060,062,780"
3,078930,2016,사업보고서,GS,715,ifrs_Revenue,매출액,"13,462,426,000,000"
4,006360,2016,사업보고서,GS건설,411,ifrs_Revenue,매출액,"11,035,596,402,664"
...,...,...,...,...,...,...,...,...
22985,005380,2016,3분기보고서,현대자동차,301,ifrs_ProfitLossBeforeTax,법인세차감전순이익,"6,039,684,000,000"
22986,011760,2016,3분기보고서,현대종합상사,468,ifrs_ProfitLossBeforeTax,법인세차감전순이익,"25,332,323,161"
22987,008770,2016,3분기보고서,호텔신라,478,ifrs_ProfitLossBeforeTax,법인세차감전순이익,"46,642,039,050"
22988,064240,2016,3분기보고서,홈캐스트,265,ifrs_ProfitLossBeforeTax,법인세차감전순이익,"12,963,640,264"


In [35]:
sample['항목코드'] = sample.apply(lambda x: 'ifrs_Revenue' if x['항목명'] == '매출액' else x['항목코드'], axis=1)
sample['항목코드'] = sample.apply(lambda x: 'ifrs_GrossProfit' if x['항목명'] == '매출총이익' else x['항목코드'], axis=1)
sample['항목코드'] = sample.apply(lambda x: 'dart_OperatingIncomeLoss' if x['항목명'] == '영업이익' else x['항목코드'], axis=1)
sample['항목코드'] = sample.apply(lambda x: 'ifrs_ProfitLoss' if x['항목명'] == '당기순이익' else x['항목코드'], axis=1)
sample['항목코드'] = sample.apply(lambda x: 'ifrs_ProfitLossAttributableToNoncontrollingInterests' if '비지배' in x['항목명'] else x['항목코드'], axis=1)
sample['항목코드'] = sample.apply(lambda x: 'dart_DepreciationExpense' if x['항목명'] == '감가상각비' else x['항목코드'], axis=1)
sample['항목코드'] = sample.apply(lambda x: 'ifrs_ProfitLossBeforeTax' if x['항목명'] == '법인세차감전순이익' else x['항목코드'], axis=1)

In [36]:
sample['항목명'] = sample.apply(lambda x: '당기순이익(비지배)' if 'ProfitLossAttributableToNoncontrollingInterests' in x['항목코드'] else x['항목명'], axis=1)

In [37]:
sample = sample.drop_duplicates()

In [46]:
sample['항목명'].unique()

array(['매출액', '매출총이익', '영업이익', '당기순이익', '당기순이익(비지배)', '감가상각비',
       '법인세차감전순이익'], dtype=object)

In [45]:
sample['항목코드'].unique()

array(['ifrs_Revenue', 'ifrs_GrossProfit', 'dart_OperatingIncomeLoss',
       'ifrs_ProfitLoss',
       'ifrs_ProfitLossAttributableToNoncontrollingInterests',
       'dart_DepreciationExpense', 'ifrs_ProfitLossBeforeTax'],
      dtype=object)

In [44]:
sample[sample['항목명'] == '당기순이익(비지배)']

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
645,001040,2016,사업보고서,CJ,715,ifrs_ProfitLossAttributableToNoncontrollingInt...,당기순이익(비지배),"346,795,361,000"
646,000120,2016,사업보고서,CJ대한통운,493,ifrs_ProfitLossAttributableToNoncontrollingInt...,당기순이익(비지배),"12,418,947,928"
647,155660,2016,사업보고서,DSR,139,ifrs_ProfitLossAttributableToNoncontrollingInt...,당기순이익(비지배),"-2,888,456"
648,078930,2016,사업보고서,GS,715,ifrs_ProfitLossAttributableToNoncontrollingInt...,당기순이익(비지배),"117,608,000,000"
649,006360,2016,사업보고서,GS건설,411,ifrs_ProfitLossAttributableToNoncontrollingInt...,당기순이익(비지배),"5,349,891,573"
...,...,...,...,...,...,...,...,...
22812,005380,2016,3분기보고서,현대자동차,301,ifrs_ProfitLossAttributableToNoncontrollingInt...,당기순이익(비지배),"244,388,000,000"
22813,011760,2016,3분기보고서,현대종합상사,468,ifrs_ProfitLossAttributableToNoncontrollingInt...,당기순이익(비지배),"96,324,233"
22814,008770,2016,3분기보고서,호텔신라,478,ifrs_ProfitLossAttributableToNoncontrollingInt...,당기순이익(비지배),NaN
22815,064240,2016,3분기보고서,홈캐스트,265,ifrs_ProfitLossAttributableToNoncontrollingInt...,당기순이익(비지배),NaN


In [42]:
손익일괄 = sample

# 저장

In [48]:
손익일괄.to_csv('/content/drive/MyDrive/22캡스톤디자인/재무일괄/일괄/손익일괄.csv')